In [1]:
# Todos as funções gráficas construídas aqui foram usadas na aplicação web app.py, que implementa
# o dashboard. Gráficos interativos não são plotados em arquivos do tipo notebook (.ipynb), logo
# não será possível visualizá-los por aqui. 
# Acesse a aplicação para vê-los: https://share.streamlit.io/joao-vitor-souza/stone-dc-2022/main/app.py

# Importando bibliotecas para leitura e manipulação de dados.
import pandas as pd
import json
from urllib.request import urlopen
from functools import reduce

# Funções gerais do sistema.
import os

# Biblioteca gráfica.
import plotly.express as px
import plotly.graph_objects as go

# Objetos temporais.
from datetime import datetime

# Filtragem de alertas.
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Importando os dados já processados.
dados = pd.read_parquet("../data/processed/geralClientesLimpo.parquet")

In [3]:
# Salvando alguns registros e a descrição geral dos dados.
head = dados.head()
describe = dados.describe(include="all")

head.to_csv("../data/curated/head.csv", index=False)
describe.to_csv("../data/curated/describe.csv")

head

,contrato_id,status_contrato,dt_vencimento,dt_contrato,dt_desembolso,vlr_desembolsado,vlr_pgto_realizado,juros_mes,flag_transacao,perc_retencao,tipo_empresa,estado,subsegmento,segmento
0,356c02706c8e74b15004bb5964ade6bb,Settled,2020-12,2020-06,2020-06,22686.57,0.0,0.10788,0,0.24,PF,PB,Educação,Serviços recorrentes
1,356c02706c8e74b15004bb5964ade6bb,Settled,2020-12,2020-06,2020-06,22686.57,0.0,0.10788,1,0.24,PF,PB,Educação,Serviços recorrentes
2,356c02706c8e74b15004bb5964ade6bb,Settled,2020-12,2020-06,2020-06,22686.57,0.0,0.10788,1,0.24,PF,PB,Educação,Serviços recorrentes
3,356c02706c8e74b15004bb5964ade6bb,Active,2020-12,2020-06,2020-06,22686.57,0.0,0.10788,0,0.24,PF,PB,Educação,Serviços recorrentes
4,356c02706c8e74b15004bb5964ade6bb,Settled,2020-12,2020-06,2020-06,22686.57,0.0,0.10788,0,0.24,PF,PB,Educação,Serviços recorrentes


In [4]:
describe

,contrato_id,status_contrato,dt_vencimento,dt_contrato,dt_desembolso,vlr_desembolsado,vlr_pgto_realizado,juros_mes,flag_transacao,perc_retencao,tipo_empresa,estado,subsegmento,segmento
count,8121276,8121276,8121276,8121276,8121276,8.121276e+06,8.121276e+06,8.121276e+06,8.121276e+06,8.121276e+06,8121276,8121276,8121276,8121276
unique,14756,6,30,19,18,NaN,NaN,NaN,NaN,NaN,3,28,16,9
top,b4d49d182c009651984e8b0bb7e2db18,Active,2022-06,2020-09,2020-09,NaN,NaN,NaN,NaN,NaN,PJ,SP,Alimentação Rápida,Alimentação
freq,1666,5686110,625879,823710,856731,NaN,NaN,NaN,NaN,NaN,4605715,2170936,1683934,2957278
mean,NaN,NaN,NaN,NaN,NaN,3.386537e+04,6.239082e+01,6.201247e-02,5.563968e-01,2.045371e-01,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,4.173352e+04,3.868506e+02,1.935896e-02,4.968093e-01,2.345014e-01,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,1.710200e+02,-4.478673e+04,5.880000e-03,0.000000e+00,1.140000e-02,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,8.721000e+03,0.000000e+00,4.788000e-02,0.000000e+00,9.600000e-02,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,1.915200e+04,0.000000e+00,5.988000e-02,1.000000e+00,1.520400e-01,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,4.126059e+04,1.925000e+01,7.188000e-02,1.000000e+00,2.040000e-01,NaN,NaN,NaN,NaN


### Gráfico de correlações

In [5]:
correlacoes = dados.corr()

correlacoes

,vlr_desembolsado,vlr_pgto_realizado,juros_mes,flag_transacao,perc_retencao
vlr_desembolsado,1.000000,0.190551,-0.255736,0.132528,0.014252
vlr_pgto_realizado,0.190551,1.000000,-0.047869,0.093914,-0.024137
juros_mes,-0.255736,-0.047869,1.000000,-0.132364,0.054830
flag_transacao,0.132528,0.093914,-0.132364,1.000000,-0.246661
perc_retencao,0.014252,-0.024137,0.054830,-0.246661,1.000000


In [6]:
# Instanciando mapa de calor.
fig = px.imshow(
    
    # Dados.
    correlacoes,
    
    # A razão de como o gráfico se ajusta à tela será automática.
    aspect="auto",
    
    # Escala de cores.
    color_continuous_scale=px.colors.diverging.BrBG
)

# Estilização.
fig.update_layout(
    
    # Margens do gráfico.
    margin=dict(l=20,r=20,b=20,t=100),
    
    # Título.
    title=go.layout.Title(
        text="Correlações Entre Algumas Grandezas Numéricas <br><sup>Um maior percentual de retenção tende a gerar menos transações</sup>"),
    
    # Tamanho da fonte.
    font=dict(size=15)
)

# Carregando para a pasta de dados prontos para consumo.
fig.write_json("../data/curated/correlacoes.json")

### Quantidade de contratos vencendo por data

In [7]:
# Agrupando os dados por contrato, selecionando a última aparição do atributo vencimento de cada contrato,
# contando a quantidade de registros por data de vencimento e resetando os índices.
qtd_contratos_venc = dados.groupby(["contrato_id"]).dt_vencimento.last().value_counts().reset_index()

# Renomeando colunas.
qtd_contratos_venc.rename({"index": "data", "dt_vencimento": "contratos"}, axis=1, inplace=True)

# Ordenando as datas de vencimento de forma crescente.
qtd_contratos_venc.sort_values(by="data", inplace=True)

qtd_contratos_venc

,data,contratos
25,2020-09,52
12,2020-10,623
13,2020-11,494
21,2020-12,115
26,2021-01,28
18,2021-03,205
29,2021-04,3
28,2021-05,3
20,2021-06,152
14,2021-07,440


In [8]:
# Instanciando figura vazia.
fig = go.Figure()

# Adicionando gráfico de linha à figura.
fig.add_trace(
    go.Scatter(
        
        # Dados no eixo x (datas de vencimento).
        x=qtd_contratos_venc.data, 
        
        # Dados no eixo y (contagem de contratos vencendo por data).
        y=qtd_contratos_venc.contratos, 
        
        # Plotando as linhas e os marcadores em cada ponto.
        mode="lines+markers",
        
        # Nome da curva.
        name="Curva de Vencimentos",
        
        # Cor dos marcadores.
        marker_color="skyblue", 
        
        # Informações que aparecerão na animação do hover.
        hovertemplate="Data Limite dos Contratos: %{x} <br>Quantidade de Contratos Vencidos: %{y}<extra></extra>"
    )
)

# Recuperando a data atual.
agora = datetime.strftime(datetime.now(), "%Y-%m")

# Adicionando ponto atual.
fig.add_trace(
    go.Scatter(
        
        # Dado no eixo x (data atual).
        x=[agora], 
        
        # Dado no eixo y (quantidade de contratos vencendo no mês atual).
        y=[int(qtd_contratos_venc[qtd_contratos_venc.data == agora].contratos.values)], 
        
        # Plotando somente o marcador.
        mode="markers", 
        
        # Cor e tamanho do marcador.
        marker=dict(size=[8], color=["orangered"]),
        
        # Nome do marcador.
        name="Estamos aqui", 
        
        # Informações que aparecerão na animação do hover.
        hovertemplate="Estamos Aqui! <br><br>Data Limite dos Contratos: %{x} <br>Quantidade de Contratos Vencidos: %{y}<extra></extra>"
    )
)

# Estilização.
fig.update_layout(
    
    # Título.
    title=go.layout.Title(
        text="Contratos que Venceram e Vencerão ao Longo dos Próximos Meses <br><sup>Estamos caminhando para o pico histórico de contratos que vencerão</sup>"),
    
    # Tamanho da fonte.
    font=dict(size=15),
    
    # Escondendo a legenda.
    showlegend=False
)

fig.write_json("../data/curated/qtd_contratos_venc.json")

### Quantidade de contratos criados por data

In [9]:
# Agrupando os dados por contrato, selecionando a primeira aparição do atributo data de criação de cada 
# contrato, contando a quantidade de registros por data e resetando os índices.
qtd_contratos_novos = dados.groupby(["contrato_id"]).dt_contrato.first().value_counts().reset_index()

# Renomeando colunas.
qtd_contratos_novos.rename({"index": "data", "dt_contrato": "contratos"}, axis=1, inplace=True)

# Ordenando as datas de criação de forma crescente.
qtd_contratos_novos.sort_values(by="data", inplace=True)

qtd_contratos_novos

,data,contratos
18,2019-12,52
12,2020-01,623
13,2020-02,489
14,2020-03,463
15,2020-04,423
16,2020-05,339
6,2020-06,975
4,2020-07,1131
5,2020-08,1087
0,2020-09,1416


In [10]:
# A adição do gráfico de linha segue a mesma lógica do gráfico anterior.
fig = go.Figure()

# Gráfico de linha.
fig.add_trace(
    go.Scatter(
        x=qtd_contratos_novos.data, 
        y=qtd_contratos_novos.contratos,
        name="Histórico",
        mode="lines+markers",
        marker_color="skyblue", 
        hovertemplate="Data de Referência ao Mês Anterior: %{x} <br>Quantidade de Contratos Feitos: %{y}<extra></extra>"
    )
)

# Nesse gráfico adicionaremos uma reta com dos valores médios.
# Obs: Adicionamos a média geral e não uma média móvel.
fig.add_trace(
    go.Scatter(
        
        # Dados no eixo x (Todas as datas).
        x=qtd_contratos_novos.data,
        
        # Dados no eixo y (A média de contratos criados).
        y=[qtd_contratos_novos.contratos.mean()] * qtd_contratos_novos.shape[0],
        
        name="Média",
        mode="lines",
        line=dict(color="red", width=3, dash="dot")
    )
)

# Estilização.
fig.update_layout(
    title=go.layout.Title(
        text="Histórico da Quantidade de Contratos Feitos <br><sup>Por quase um ano ficamos acima da média em quantidade de contratos assinados</sup>"),
    font=dict(size=15)
)

fig.write_json("../data/curated/qtd_contratos_novos.json")

### Valor desembolsado pela Stone em cada mês

In [11]:
# Agrupando os dados por contrato, selecionando a primeira aparição dos atributos data de desembolso e
# valor desembolsado de cada contrato.
qtd_vlr_des = dados.groupby(["contrato_id"])["dt_desembolso", "vlr_desembolsado"].first()

# Agrupando os dados por data de desembolso, somando a coluna valor desembolsado e resetando os índices.
qtd_vlr_des = qtd_vlr_des.groupby("dt_desembolso").sum().reset_index()

qtd_vlr_des

,dt_desembolso,vlr_desembolsado
0,2020-01,10679828.96
1,2020-02,12828066.63
2,2020-03,15248785.99
3,2020-04,8992053.81
4,2020-05,7615529.46
5,2020-06,21871864.44
6,2020-07,30347450.39
7,2020-08,41844951.72
8,2020-09,58386664.18
9,2020-10,33195858.93


In [12]:
# A construção desse gráfico segue a mesma lógica dos outros dois.

fig = go.Figure()

# Gráfico de linha.
fig.add_trace(
    go.Scatter(
        x=qtd_vlr_des.dt_desembolso, 
        y=qtd_vlr_des.vlr_desembolsado,
        name="Histórico",
        mode="lines+markers",
        marker_color="skyblue", 
        hovertemplate="Mês de Referência do Desembolso: %{x} <br>Valor Total Desembolsado: %{y}<extra></extra>"
    )
)

# Média.
fig.add_trace(
    go.Scatter(
        x=qtd_vlr_des.dt_desembolso, 
        y=[qtd_vlr_des.vlr_desembolsado.mean()] * qtd_vlr_des.shape[0], 
        name="Média",
        mode="lines",
        line=dict(color="red", width=3, dash="dot"), 
        hovertemplate="Média: %{y}<extra></extra>"
    )
)

datasCrescimento = ["2020-05", "2020-06", "2020-07", "2020-08", "2020-09"]

# Linha de crescimento.
fig.add_trace(
    go.Scatter(
        x=datasCrescimento,
        y=qtd_vlr_des[qtd_vlr_des.dt_desembolso.isin(datasCrescimento)].vlr_desembolsado, 
        name="Crescimento",
        mode="lines+markers",
        hovertemplate="Mês de Referência do Desembolso: %{x} <br>Valor Total Desembolsado: %{y}<extra></extra>"
    )
)

# Estilização.
fig.update_layout(
    title=go.layout.Title(
        text="Valor Total Desembolsado (em R$) Pela Stone por Meses <br><sup>No primeiro ano de pandemia houve um crescimento vertiginoso do valor emprestado pela Stone</sup>"), 
    font=dict(size=15)
)

fig.write_json("../data/curated/qtd_vlr_des.json")

### Distribuição dos segmentos por valor de pagamento realizado

In [13]:
# Agrupando os dados por contrato e segmento, somando parcialmente o atributo de valor de pagamento 
# realizado e resetando os índices.
qtd_vlr_pgt = dados.groupby(["segmento", "contrato_id"])["vlr_pgto_realizado"].sum().reset_index()

# Agrupando por segmento, somando o atributo de valor de pagamento e resetando os índices novamente.
qtd_vlr_pgt = qtd_vlr_pgt.groupby("segmento").sum().reset_index()

# Ordenando por valor de pagamento realizado de forma descrescente.
qtd_vlr_pgt.sort_values(by="vlr_pgto_realizado", inplace=True, ascending=False)

qtd_vlr_pgt

,segmento,vlr_pgto_realizado
0,Alimentação,1.903374e+08
1,Bens duráveis,8.839530e+07
7,Varejo,8.294153e+07
6,Supermercado/Farmácia,6.070926e+07
4,Serviços,5.545592e+07
5,Serviços recorrentes,9.405358e+06
3,Posto,8.293325e+06
8,Viagens e entretenimento,6.597528e+06
2,Outros,4.557470e+06


In [14]:
# Instaciando gráfico de barras.
fig = go.Figure(
    data=[
        go.Bar(
            
            # Dados no eixo x (Nome de cada segmento).
            x=qtd_vlr_pgt.segmento,
            
            # Dados no eixo y (Valor do pagamento realizado).
            y=qtd_vlr_pgt.vlr_pgto_realizado,
            
            # Cores de cada barra.
            marker_color = ["crimson"] + ["lightslategray"] * 8
        )
    ]
)

# Estilização.
fig.update_layout(
    title=go.layout.Title(
        text="Valores Totais Pagos (em R$) por Cada Segmento <br><sup>Alimentação é maior que os segmentos de Bens Duráveis, Varejo, Serviços Recorrentes e Viajens somados</sup>"),
    font=dict(size=14)
)

fig.write_json("../data/curated/qtd_vlr_pgt.json")

### Distribuição relativa dos subsegmentos

In [15]:
# Agrupando os dados por contrato, segmento e subsegmento, selecionando as colunas de segmentos e subsegmentos
# e fazendo uma amostra de cada contrato.
qtd_sub_count = dados.groupby(["segmento", 
                               "subsegmento", 
                               "contrato_id"], 
                              group_keys=False)["segmento", 
                                                "subsegmento"].apply(lambda df: df.sample(1))

# Foram identificados alguns valores None. Eles serão convertidos para o subsegmento Outros.
qtd_sub_count.replace({"None": "Outros"}, inplace=True)

# Agrupando por segmento e fazendo a contagem dos subsegmentos.
qtd_sub_count = qtd_sub_count.groupby("segmento").value_counts().reset_index()

# Renomeando a coluna recém criada "0" para "frequencia".
qtd_sub_count.rename({0: "frequecia"}, axis=1, inplace=True)

# Adicionando a coluna de frequência relativa.
qtd_sub_count["frequecia_pct"] = qtd_sub_count["frequecia"] / qtd_sub_count.groupby(["segmento"])["frequecia"].transform(sum) * 100

qtd_sub_count

,segmento,subsegmento,frequecia,frequecia_pct
0,Alimentação,Alimentação Rápida,3069,56.854391
1,Alimentação,Bares e Restaurantes,1271,23.545758
2,Alimentação,Lojas Diversas,610,11.300482
3,Alimentação,Supermercados,397,7.354576
4,Alimentação,Atacadista de Alimento,51,0.944794
5,Bens duráveis,Automotivo,584,25.369244
6,Bens duráveis,Lojas Diversas,581,25.238923
7,Bens duráveis,Material de Construção,556,24.152911
8,Bens duráveis,Outros,250,10.860122
9,Bens duráveis,Saúde,205,8.905300


In [16]:
# Instanciando um gráfico sunburst.
fig = px.sunburst(
    
    # Dados a serem plotados.
    qtd_sub_count, 
    
    # Os caminhos para os dados pais e filhos, respectivamente.
    path=["segmento", "subsegmento"],
    
    # Valores de referência da distribuição.
    values="frequecia", 
    
    # Dimensões.
    width=750,
    height=750,
    
    # Título da caixa animada do hover.
    hover_name = "segmento",
    
    # Dados customizados para criação do hovertemplate completo.
    custom_data = qtd_sub_count.columns
)

# Estilização.
fig.update_layout(
    margin=dict(l=0,r=0,b=0,t=150),
    uniformtext=dict(minsize=13, mode="hide"),
    title=go.layout.Title(
        text="Distribuição Relativa dos Subsegmentos <br><sup>Talvez contra-intuitivamente, Alimentação Rápida é muito maior que o subsegmento de Supermercados</sup>"),
    font=dict(size=14)
)

# Adicionando a orientação dos rótulos e o hovertemplate.
fig.update_traces(
    go.Sunburst(insidetextorientation="radial"),
    hovertemplate="<br>".join([
        "Segmento: %{customdata[0]}",
        "Subsegmento: %{customdata[1]}",
        "Frequência Absoluta: %{customdata[2]}",
        "Frequência Relativa: %{customdata[3]:.2f}%",
    ]) 
)

fig.write_json("../data/curated/qtd_sub_count.json")

### Valores e Contratos Ativos por Estado

In [17]:
# Carregando os dados geoespaciais de uma fonte remota.
with urlopen('https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/brazil-states.geojson') as response:
    mapa = json.load(response)

# Salvando os dados localmente.
with open('../data/raw/mapa.json', 'w') as arq:
    json.dump(mapa, arq)

In [18]:
# Função para retornar o índice dos dados agrupados.
def retorna_index(group):
    return group.index

# Filtrando os estados diferentes de ND e contratos ativos, agrupando por estado e contrato, 
# retornando o índice do agrupamento.
qtd_contratos_estado = dados[
    (dados.estado != "ND") & (dados.status_contrato == "Active")
].groupby(["estado", "contrato_id"]).apply(retorna_index)

# Contando a quantidade de contratos por estado.
qtd_contratos_estado = qtd_contratos_estado.groupby("estado").count().reset_index()

# Renomeando coluna.
qtd_contratos_estado.rename({0: "contratos"}, axis=1, inplace=True)

# Filtrando os estados diferentes de ND e contratos ativos, agrupando por estado e contrato, 
# retornando a primeira aparição do atributo valor desembolsado de cada contrato.
qtd_vlr_des_estado = dados[
    (dados.estado != "ND") & (dados.status_contrato == "Active")
].groupby(["estado", "contrato_id"]).vlr_desembolsado.first().reset_index()

# Somando o valor desembolsado por estado.
qtd_vlr_des_estado = qtd_vlr_des_estado.groupby("estado").vlr_desembolsado.sum().reset_index()

# Dividindo por 1 milhão.
qtd_vlr_des_estado["vlr_desembolsado"] = qtd_vlr_des_estado.vlr_desembolsado/1e6

# Filtrando os estados diferentes de ND e contratos ativos, agrupando por estado e contrato, 
# fazendo a soma do atributo valor do pagamento realizado de cada contrato.
qtd_vlr_pgt_estado = dados[
    (dados.estado != "ND") & (dados.status_contrato == "Active")
].groupby(["estado", "contrato_id"]).vlr_pgto_realizado.sum().reset_index()

# Somando o valor do pagamento de cada contrato, por estado.
qtd_vlr_pgt_estado = qtd_vlr_pgt_estado.groupby("estado").vlr_pgto_realizado.sum().reset_index()

# Dividindo por 1 milhão.
qtd_vlr_pgt_estado["vlr_pgto_realizado"] = qtd_vlr_pgt_estado.vlr_pgto_realizado/1e6

# Fazendo o merge de todos os dataframes.
dfs = [qtd_contratos_estado, qtd_vlr_des_estado, qtd_vlr_pgt_estado]
qtds_mapa = reduce(lambda df1, df2: pd.merge(df1, df2, on=["estado"]), dfs)

qtds_mapa

,estado,contratos,vlr_desembolsado,vlr_pgto_realizado
0,AC,34,1.095756,0.983544
1,AL,147,5.445676,5.027795
2,AM,144,4.565515,4.162917
3,AP,70,3.298125,3.133058
4,BA,816,27.394954,24.550251
5,CE,435,13.386199,12.626110
6,DF,521,20.898848,19.149321
7,ES,312,7.837232,7.461823
8,GO,585,20.174279,19.075217
9,MA,161,5.960811,5.103306


In [19]:
# Fazendo o mapeamento entre os dados geoespaciais e os dados sumarizados usando a sigla de cada estado.
mapping = {}
for i in range(27):
    sigla_estado = mapa["features"][i]["properties"]["sigla"]
    mapa["features"][i]["id"] = sigla_estado
    mapping[sigla_estado] = mapa["features"][i]["properties"]["name"]

# Criando uma coluna com o nome dos estados.
qtds_mapa["nome_estado"] = qtds_mapa["estado"].map(mapping)

In [20]:
# Salvando os dados geoespaciais processados.
with open('../data/processed/mapaProcessado.json', 'w') as arq:
    json.dump(mapa, arq)

In [21]:
# Instanciando mapa.
fig = px.choropleth(
    
    # Dados sumarizados.
    qtds_mapa,
    
    # Dados geoespaciais.
    geojson=mapa, 
    
    # Propriedade mapeada. No caso estamos usando a sigla de cada estado.
    locations='estado', 
    
    # As cores se basearão na quantidade de contratos ativos no estado.
    color='contratos',
    
    # Escala de cores.
    color_continuous_scale="mint",
    
    # Dados de referência para o efeito hover.
    custom_data = qtds_mapa.columns,
    
    # Renomeando o título da barra de cores.
    labels={'contratos': 'Contratos'}
)

# Estilizações.
fig.update_layout(margin={"r":0,"l":0,"b":0},
                  title=go.layout.Title(
                     text="Valores e Contratos Ativos por Estado<br><sup>São Paulo tem mais contratos do que o Norte, Nordeste, Mato Grosso e Mato Grosso do Sul somados</sup>"),
                  font=dict(size=14), 
                  geo=dict(bgcolor= 'rgba(0,0,0,0)'))
fig.update_geos(fitbounds = "locations", visible = False)
fig.update_traces(
    hovertemplate="<br>".join(
        [
            "<b>%{customdata[4]}</b><br>",
             "Contratos Ativos: %{customdata[1]}",
             "Valor Desembolsado Total: R$ %{customdata[2]:.3E} milhões",
             "Valores Pagos Total: R$ %{customdata[3]:.3E} milhões",
        ]
    )
)

fig.write_json("../data/curated/qtds_mapa.json")

### Exemplo de Análise Sazonal

In [22]:
# Alterando o cwd para a pasta do modelo.
os.chdir(os.path.join(os.getcwd(), "../models"))

In [23]:
# Importando as funções de utilidade do modelo.
from utils.gerador import Gerador
from utils.plots import Plots

In [24]:
# Carregando os dados processados para análise sazonal.
dados = pd.read_parquet("../data/processed/geralTpvLimpo.parquet")

In [ ]:
# Gerando dados sazonais.
dados_modelo = Gerador().dadosSazonalidade(dados, contrato_id="6fad93bf0fe4314745fe79a4d2d42277", status_ativo="Active")

# Gerando curvas sazonais.
grafico = Plots(dados_modelo)
grafico.gerarCurvaSazonalidade()
sazonalidade, sazonalidadeComponentes = grafico.graficos

sazonalidade.write_json("../data/curated/sazonalidade.json")
sazonalidadeComponentes.write_json("../data/curated/sazonalidadeComponentes.json")